# test to load local package

### __test 1__
 - --init--.py are empty
     - only works with parent.shild.grandchild etc

### __test 2__
 - --init--.py contain from module import class
     - works with from module import class

In [4]:
# from SPOTS_main import Post_processing
# from SPOTS_main import plot
# from SPOTS_main import TS_processing


import SPOTSAR_main as sm

sm.Post_processing.reorder_files()
print(dir(sm.plot))
print(dir(sm.TS_processing))


TypeError: reorder_files() missing 2 required positional arguments: 'dir' and 'pattern'